In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime

In [2]:
df = pd.read_csv('Reg_Gas.csv')

In [3]:
df

,Location,Month,Prices
0,"St. John's, Newfoundland and Labrador",Jan-90,57.7
1,"St. John's, Newfoundland and Labrador",Feb-90,58.2
2,"St. John's, Newfoundland and Labrador",Mar-90,58.5
3,"St. John's, Newfoundland and Labrador",Apr-90,59.3
4,"St. John's, Newfoundland and Labrador",May-90,59.8
...,...,...,...
7410,"Yellowknife, Northwest Territories",Nov-23,168.7
7411,"Yellowknife, Northwest Territories",Dec-23,163.4
7412,"Yellowknife, Northwest Territories",Jan-24,157.4
7413,"Yellowknife, Northwest Territories",Feb-24,157


In [4]:
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y')

In [5]:
df['Year'] = df['Month'].dt.year
df['Month_Num'] = df['Month'].dt.month

In [6]:
df = df[pd.to_numeric(df['Prices'], errors='coerce').notnull()]

In [7]:
df['Prices'] = df['Prices'].astype(float)

C:\Users\jack\AppData\Local\Temp\ipykernel_32108\512960355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prices'] = df['Prices'].astype(float)


In [8]:
groups = df.groupby('Location')
df

,Location,Month,Prices,Year,Month_Num
0,"St. John's, Newfoundland and Labrador",1990-01-01,57.7,1990.0,1.0
1,"St. John's, Newfoundland and Labrador",1990-02-01,58.2,1990.0,2.0
2,"St. John's, Newfoundland and Labrador",1990-03-01,58.5,1990.0,3.0
3,"St. John's, Newfoundland and Labrador",1990-04-01,59.3,1990.0,4.0
4,"St. John's, Newfoundland and Labrador",1990-05-01,59.8,1990.0,5.0
...,...,...,...,...,...
7410,"Yellowknife, Northwest Territories",2023-11-01,168.7,2023.0,11.0
7411,"Yellowknife, Northwest Territories",2023-12-01,163.4,2023.0,12.0
7412,"Yellowknife, Northwest Territories",2024-01-01,157.4,2024.0,1.0
7413,"Yellowknife, Northwest Territories",2024-02-01,157.0,2024.0,2.0


In [9]:
predicted_df = pd.DataFrame()


In [10]:
def predict_for_group(group):  # Define a function for clarity
    # Splitting the data into features (X) and target variable (y)
    X = group[['Year', 'Month_Num']]
    y = group['Prices']

    # Training a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Create a DataFrame for future dates for this location
    max_date = group['Month'].dt.to_period('M').max().to_timestamp('M')  
    next_month_start = max_date + pd.offsets.MonthBegin(1)  
    future_dates = pd.date_range(start=next_month_start, periods=60, freq='M')  

    future_df = pd.DataFrame({'Month': future_dates})
    future_df['Year'] = future_df['Month'].dt.year
    future_df['Month_Num'] = future_df['Month'].dt.month

    # Make predictions for this location
    future_predictions = model.predict(future_df[['Year', 'Month_Num']])

    # Add predictions to the predicted DataFrame
    future_df['Predicted Prices'] = future_predictions
    future_df['Location'] = name

    return future_df  # Return the predictions for this group

# Apply changes to all groups:
predicted_df = pd.DataFrame()  # Initialize empty DataFrame
for name, group in groups:
    predictions = predict_for_group(group)
    predicted_df = pd.concat([predicted_df, predictions])


In [11]:
# Round predicted prices to 1 decimal place
predicted_df['Predicted Prices'] = predicted_df['Predicted Prices'].round(1)
predicted_df['Month'] = predicted_df['Month_Num']
predicted_df.drop('Month_Num', axis=1, inplace=True)
predicted_df.reset_index(drop=True, inplace=True)
predicted_df.head(12)

,Month,Year,Predicted Prices,Location
0,4,2024,133.0,"Calgary, Alberta"
1,5,2024,133.3,"Calgary, Alberta"
2,6,2024,133.6,"Calgary, Alberta"
3,7,2024,134.0,"Calgary, Alberta"
4,8,2024,134.3,"Calgary, Alberta"
5,9,2024,134.6,"Calgary, Alberta"
6,10,2024,135.0,"Calgary, Alberta"
7,11,2024,135.3,"Calgary, Alberta"
8,12,2024,135.6,"Calgary, Alberta"
9,1,2025,134.7,"Calgary, Alberta"


In [12]:
predicted_df.to_csv('Predicted_Reg_Gas_Prices.csv', index=False)